<a href="https://colab.research.google.com/github/taninnguk/DADS5001-6720422009/blob/main/HW2_premier_league_plotly_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Premier League — Overall vs Home/Away (Plotly)

In [ ]:
!pip install selenium
!apt-get update
!apt-get install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 15.0 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,414 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubu

In [ ]:
# If you're in Colab, these are already available.
# !pip -q install plotly pandas numpy

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from selenium import webdriver
pd.options.display.float_format = "{:,.2f}".format

## 1) Load data

In [ ]:
# # Point these to your files.
# DEFAULT_OVERALL = 'premier_league_table_overall.csv'
# DEFAULT_HOMEAWAY = 'premier_league_table_home_away.csv'

# def ensure_files():
#     p_overall = Path(DEFAULT_OVERALL)
#     p_ha = Path(DEFAULT_HOMEAWAY)

#     if p_overall.exists() and p_ha.exists():
#         return p_overall, p_ha

#     # In Colab, let user upload if not found.
#     try:
#         from google.colab import files  # type: ignore
#         print("⬆️ Please upload the two CSV files…")
#         uploaded = files.upload()
#         # Try to map names if different
#         for name in uploaded.keys():
#             if 'overall' in name.lower():
#                 p_overall = Path(name)
#             if 'home' in name.lower() and 'away' in name.lower():
#                 p_ha = Path(name)
#         return p_overall, p_ha
#     except Exception:
#         # Fallback: try known paths from this chat session
#         chat_overall = Path('/mnt/data/premier_league_table_overall.csv')
#         chat_ha = Path('/mnt/data/premier_league_table_home_away.csv')
#         if chat_overall.exists() and chat_ha.exists():
#             return chat_overall, chat_ha
#         raise FileNotFoundError("Could not locate CSVs. Upload them or set the paths.")

# p_overall, p_ha = ensure_files()
# print("Using:", p_overall, "and", p_ha)

# overall = pd.read_csv(p_overall)
# ha = pd.read_csv(p_ha)

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    return driver

def tidy_home_away(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # 1) flatten MultiIndex -> "Home_Pts" / "Away_Pts" ฯลฯ
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join([str(x) for x in tup if pd.notna(x) and str(x).strip() != ""]).strip("_")
            for tup in df.columns
        ]
    df.columns = [c.replace("\xa0"," ").strip() for c in df.columns]

    # 2) มาตรฐานชื่อคอลัมน์
    rename = {
        "Squad": "Team",
        "Rk": "Rk",
        "Home_MP": "MP_home",         "Away_MP": "MP_away",
        "Home_W": "W_home",           "Away_W": "W_away",
        "Home_D": "D_home",           "Away_D": "D_away",
        "Home_L": "L_home",           "Away_L": "L_away",
        "Home_GF": "GF_home",         "Away_GF": "GF_away",
        "Home_GA": "GA_home",         "Away_GA": "GA_away",
        "Home_GD": "GD_home",         "Away_GD": "GD_away",
        "Home_Pts": "Pts_home",       "Away_Pts": "Pts_away",
        "Home_Pts/MP": "Pts/MP_home", "Away_Pts/MP": "Pts/MP_away",
        "Home_xG": "xG_home",         "Away_xG": "xG_away",
        "Home_xGA": "xGA_home",       "Away_xGA": "xGA_away",
        "Home_xGD": "xGD_home",       "Away_xGD": "xGD_away",
        "Home_xGD/90": "xGD/90_home", "Away_xGD/90": "xGD/90_away",
    }
    df.rename(columns=rename, inplace=True)

    # 3) แปลงตัวเลข
    for c in df.columns:
        if c not in ["Team"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

driver = web_driver()
data=[]
driver.get('https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691')
data = driver.page_source

df_overall = pd.read_html(data, attrs={"id":"results2025-202691_overall"})
df_overall = df_overall[0]
df_home_away = pd.read_html(data, attrs={"id":"results2025-202691_home_away"})
df_home_away = df_home_away[0]
overall = df_overall
# ใช้กับ DataFrame ที่อ่านมาจาก fbref
ha = tidy_home_away(df_home_away)

# เช็คว่าคอลัมน์มาแล้ว
print([c for c in ha.columns if c.endswith("_home") or c.endswith("_away")][:12])

/tmp/ipython-input-2592017031.py:55: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_overall = pd.read_html(data, attrs={"id":"results2025-202691_overall"})
/tmp/ipython-input-2592017031.py:57: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_home_away = pd.read_html(data, attrs={"id":"results2025-202691_home_away"})


['MP_home', 'W_home', 'D_home', 'L_home', 'GF_home', 'GA_home', 'GD_home', 'Pts_home', 'Pts/MP_home', 'xG_home', 'xGA_home', 'xGD_home']


## 2) Clean & standardize

In [ ]:
# Drop accidental unnamed index columns if present
for df in (overall, ha):
    to_drop = [c for c in df.columns if str(c).lower().startswith('unnamed:')]
    if to_drop:
        df.drop(columns=to_drop, inplace=True, errors='ignore')

# Standardize key column names
overall.rename(columns={
    'Squad': 'Team',
    'Pts/MP': 'Pts_per_MP',
    'xGD/90': 'xGD_per_90'
}, inplace=True)

# Convert numeric-ish columns
def _to_numeric_cols(df, exclude=('Team', 'Goalkeeper', 'Top Team Scorer', 'Last 5', 'Notes')):
    for c in df.columns:
        if c in exclude:
            continue
        df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

overall = _to_numeric_cols(overall)

# Home/Away already suffixed in this dataset; standardize name
ha.rename(columns={'Squad': 'Team'}, inplace=True)
ha = _to_numeric_cols(ha, exclude=('Team',))

# Derived/Helper fields
def recent_form_points(s: pd.Series) -> pd.Series:
    # Map Last 5 like "D W W W W"
    m = {'W': 3, 'D': 1, 'L': 0}
    def calc(x):
        if pd.isna(x): return np.nan
        toks = [t for t in str(x).replace(',', ' ').split() if t in m]
        return sum(m[t] for t in toks[:5])
    return s.apply(calc)

overall['Pts_Last_5_match'] = recent_form_points(overall['Last 5'])

# Attendance to numeric (remove commas)
if 'Attendance' in overall.columns:
    overall['Attendance'] = (
        overall['Attendance'].astype(str).str.replace(',', '', regex=False).astype(float)
    )

# Sanity: merge keys we'll use later
key_cols = ['Team', 'Rk']
overall_key = overall[key_cols].copy()

## 3) Visualizations (Plotly)

In [ ]:
# 3.1 Overall Points (horizontal bar)
df_pts = overall[['Team','Pts']].sort_values('Pts', ascending=True)
fig1 = px.bar(df_pts, x='Pts', y='Team', orientation='h', text='Pts',
              title='Premier League Points (Overall)')
fig1.update_layout(height=650, xaxis_title='Points', yaxis_title='')
fig1.update_traces(textposition='outside', cliponaxis=False)
fig1.show()

In [ ]:
# 3.2 xGD vs Points (bubble by GF)
hover_cols = ['MP','W','D','L','GF','GA','GD','Pts_per_MP','xG','xGA','xGD_per_90','Pts_Last_5_match']
use_cols = [c for c in hover_cols if c in overall.columns]

fig2 = px.scatter(
    overall, x='xGD', y='Pts', size='GF' if 'GF' in overall.columns else None,
    hover_name='Team', hover_data=use_cols,
    title='xGD vs Points (bubble = GF)'
)
fig2.update_layout(height=550)
fig2.add_hline(y=overall['Pts'].mean(), line_dash='dot', annotation_text='Avg Pts', annotation_position='bottom right')
fig2.add_vline(x=overall['xGD'].mean(), line_dash='dot', annotation_text='Avg xGD', annotation_position='top left')
fig2.show()

In [ ]:
# --- เตรียม ha ให้พล็อตได้ ---
import pandas as pd
from bs4 import BeautifulSoup, Comment

def prepare_ha_for_plot(df: pd.DataFrame):
    g = df.copy()

    # 1) flatten MultiIndex columns -> "Home_Pts", "Away_Pts", ...
    if isinstance(g.columns, pd.MultiIndex):
        g.columns = [
            "_".join([str(x) for x in t if pd.notna(x) and str(x).strip() != ""]).strip("_")
            for t in g.columns
        ]
    g.columns = [c.replace("\xa0"," ").strip() for c in g.columns]

    # ตัดคอลัมน์ Unnamed ทิ้ง
    g = g[[c for c in g.columns if not c.startswith("Unnamed")]]

    # 2) ให้แน่ใจว่ามี 'Team'
    if 'Team' not in g.columns:
        if 'Squad' in g.columns:
            g = g.rename(columns={'Squad':'Team'})
        else:
            # ถ้ายังไม่มี เดี๋ยวไปดึงจาก HTML ภายหลัง
            g.insert(0, 'Team', pd.NA)

    # 3) รีเนมคอลัมน์คะแนนให้เป็นมาตรฐาน
    rename = {
        "Home_Pts": "Pts_home", "Away_Pts": "Pts_away",
        "Home_Pts/MP": "Pts/MP_home", "Away_Pts/MP": "Pts/MP_away",
        "Home_MP": "MP_home", "Away_MP": "MP_away",
        "Home_W": "W_home", "Away_W": "W_away",
        "Home_D": "D_home", "Away_D": "D_away",
        "Home_L": "L_home", "Away_L": "L_away",
        "Home_GF": "GF_home", "Away_GF": "GF_away",
        "Home_GA": "GA_home", "Away_GA": "GA_away",
        "Home_GD": "GD_home", "Away_GD": "GD_away",
        "Home_xG": "xG_home", "Away_xG": "xG_away",
        "Home_xGA": "xGA_home", "Away_xGA": "xGA_away",
        "Home_xGD": "xGD_home", "Away_xGD": "xGD_away",
        "Home_xGD/90": "xGD/90_home", "Away_xGD/90": "xGD/90_away",
    }
    g.rename(columns=rename, inplace=True)

    # 4) บังคับเป็นตัวเลข
    for c in g.columns:
        if c != "Team":
            g[c] = pd.to_numeric(g[c], errors="coerce")

    return g

def fill_team_from_html(page_html: str, table_id: str, g: pd.DataFrame) -> pd.DataFrame:
    """เติมชื่อทีมจาก HTML ของตาราง fbref (รองรับที่อยู่ในคอมเมนต์ด้วย)"""
    if 'Team' in g.columns and g['Team'].notna().any():
        return g  # มีแล้วไม่ต้องเติม

    soup = BeautifulSoup(page_html, "lxml")
    table = soup.find("table", id=table_id)
    if table is None:
        for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
            if table_id in c:
                table = BeautifulSoup(c, "lxml").find("table", id=table_id)
                break
    if table is None:
        return g  # หาไม่เจอ ก็ปล่อยไว้ก่อน

    # fbref บางหน้าเก็บทีมใน td[data-stat="team"] หรือ th[data-stat="team"]
    teams = [a.get_text(strip=True) for a in table.select('tbody tr td[data-stat="team"] a')]
    if not teams:
        teams = [a.get_text(strip=True) for a in table.select('tbody tr th[data-stat="team"] a')]

    if teams and len(teams) >= len(g):
        g = g.copy()
        g['Team'] = teams[:len(g)]
    return g

# ----- ใช้งานกับ ha (หรือ df_home_away) -----
# กรณีนี้ใช้กับ ha ได้เลย
ha_ready = prepare_ha_for_plot(ha)

# ถ้า Team ยังว่าง เติมจาก HTML page_source ที่เก็บไว้ในตัวแปร data (จาก Selenium driver.page_source)
ha_ready = fill_team_from_html(data, "results2025-202691_home_away", ha_ready)

# เช็คชื่อคอลัมน์ home/away ที่ใช้พล็อต
hcol, acol = "Pts_home", "Pts_away"
assert hcol in ha_ready.columns and acol in ha_ready.columns, ha_ready.columns.tolist()
assert 'Team' in ha_ready.columns, "Team column still missing"


In [ ]:
# 3.3 Home vs Away points (grouped bar)
pts_hw = ha_ready[['Team', hcol, acol]] \
    .rename(columns={hcol:'Pts_home', acol:'Pts_away'}) \
    .sort_values(['Pts_home','Pts_away'], ascending=[False, False])

fig3 = px.bar(
    pts_hw, y='Team', x=['Pts_home','Pts_away'], barmode='group',
    title='Points: Home vs Away'
)
fig3.update_layout(height=750, xaxis_title='Points', yaxis_title='')
fig3.show()

In [ ]:
ha_ready

,Team,MP_home,W_home,D_home,L_home,GF_home,GA_home,GD_home,Pts_home,Pts/MP_home,...,L_away,GF_away,GA_away,GD_away,Pts_away,Pts/MP_away,xG_away,xGA_away,xGD_away,xGD/90_away
0,Arsenal,5,4,1,0,12,1,11,13,2.60,...,1,6,2,4,12,2.40,7.90,3.50,4.50,0.89
1,Bournemouth,5,4,1,0,8,2,6,13,2.60,...,1,8,9,-1,5,1.25,6.00,8.50,-2.50,-0.63
2,Liverpool,5,4,0,1,10,5,5,12,2.40,...,3,8,9,-1,6,1.20,9.50,7.80,1.70,0.34
3,Tottenham,5,1,1,3,5,5,0,4,0.80,...,0,12,3,9,13,2.60,5.80,6.90,-1.00,-0.21
4,Chelsea,5,2,1,2,6,6,0,7,1.40,...,1,12,5,7,10,2.00,9.10,6.20,2.90,0.57
5,Sunderland,4,3,1,0,8,2,6,10,2.50,...,2,3,5,-2,7,1.40,4.70,7.00,-2.30,-0.46
6,Manchester Utd,5,4,0,1,11,6,5,12,2.40,...,2,6,10,-4,5,1.00,7.40,10.90,-3.50,-0.70
7,Manchester City,4,3,0,1,10,3,7,9,2.25,...,2,7,4,3,7,1.40,7.10,5.20,1.90,0.38
8,Crystal Palace,5,2,3,0,8,5,3,9,1.80,...,2,6,4,2,7,1.40,7.40,6.20,1.20,0.24
9,Brighton,5,3,2,0,10,5,5,11,2.20,...,3,7,10,-3,4,0.80,7.20,6.70,0.60,0.12


In [ ]:
# 3.4 Efficiency: Pts/MP Home vs Away (1 dot per club)
eff = ha_ready[['Team','Pts/MP_home','Pts/MP_away']].copy()
eff.rename(columns={'Pts/MP_home':'Pts_per_MP_home','Pts/MP_away':'Pts_per_MP_away'}, inplace=True)

fig4 = px.scatter(
    eff, x='Pts_per_MP_home', y='Pts_per_MP_away', text='Team',
    title='Pts/MP: Home vs Away'
)
lim = max(eff[['Pts_per_MP_home','Pts_per_MP_away']].max()) + 0.25
fig4.add_shape(type='line', x0=0, y0=0, x1=lim, y1=lim, line=dict(dash='dot'))
fig4.update_traces(textposition='top center')
fig4.update_layout(height=650, xaxis_title='Home Pts/MP', yaxis_title='Away Pts/MP')
fig4.show()

In [ ]:
# 3.5 Team profile: compare Home vs Away on a few metrics (dropdown)
metrics = ['Pts/MP', 'xG', 'xGA', 'xGD', 'xGD/90']
home_cols = [m + '_home' for m in metrics]
away_cols = [m + '_away' for m in metrics]

# Build a tidy long-form frame for polar
long_rows = []
for _, row in ha_ready.iterrows():
    for side, cols in [('Home', home_cols), ('Away', away_cols)]:
        for metric, col in zip(metrics, cols):
            long_rows.append({
                'Team': row['Team'],
                'Side': side,
                'Metric': metric,
                'Value': row[col] if col in ha.columns else np.nan,
            })
tidy = pd.DataFrame(long_rows)

teams = sorted(tidy['Team'].unique().tolist())

# Default to first team
default_team = teams[0] if teams else None
df0 = tidy[tidy['Team'] == default_team]

fig5 = px.line_polar(
    df0, r='Value', theta='Metric', color='Side', line_close=True,
    title=f'Team Profile (Home vs Away): {default_team}'
)
fig5.update_traces(fill='toself')
# Add dropdown to switch teams
buttons = []
for t in teams:
    df_t = tidy[tidy['Team'] == t]
    # For each team we need two traces (Home/Away)
    df_home = df_t[df_t['Side']=='Home']
    df_away = df_t[df_t['Side']=='Away']
    buttons.append(dict(
        label=t,
        method='update',
        args=[
            {'r':[df_home['Value'], df_away['Value']], 'theta':[df_home['Metric'], df_away['Metric']]},
            {'title': f'Team Profile (Home vs Away): {t}'}
        ]
    ))
fig5.update_layout(
    updatemenus=[dict(type='dropdown', x=0.5, xanchor='center', y=1.15, buttons=buttons)]
)
fig5.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

summary = overall[['Team','Rk','Pts','GD','xGD','Pts_per_MP','Pts_Last_5_match']].sort_values('Rk')

n = len(summary)
# zebra stripes
stripe = ['#ffffff' if i % 2 == 0 else '#f7f9fc' for i in range(n)]
fill_colors = [stripe[:] for _ in summary.columns]

# ไฮไลท์ Top4 / Bottom3 ตาม Rk
top4 = set(summary.nsmallest(4, 'Rk').index)
bottom3 = set(summary.nlargest(3, 'Rk').index)
for j, _col in enumerate(summary.columns):
    for i, idx in enumerate(summary.index):
        if idx in top4:    fill_colors[j][i] = '#e9fbe7'   # เขียวอ่อน
        if idx in bottom3: fill_colors[j][i] = '#fde7e7'   # แดงอ่อน

# จัดรูปแบบตัวเลขต่อคอลัมน์
fmt = []
for c in summary.columns:
    if c in ['Rk','Pts','Pts_Last_5_match','GD']: fmt.append('d')
    elif c in ['xGD']:                             fmt.append('+.2f')
    elif c in ['Pts_per_MP']:                      fmt.append('.2f')
    else:                                          fmt.append(None)

fig6 = go.Figure(data=[go.Table(
    columnwidth=[180, 60, 60, 60, 80, 100, 130],
    header=dict(
        values=list(summary.columns),
        align='center',
        fill_color='#111827',
        font=dict(color='white', size=13),
        height=34
    ),
    cells=dict(
        values=[summary[c] for c in summary.columns],
        align=['left'] + ['center']*(len(summary.columns)-1),
        fill_color=fill_colors,
        line_color='#e5e7eb', line_width=1,
        height=28,
        format=fmt
    )
)])
fig6.update_layout(title='Summary — Overall Table', margin=dict(l=0,r=0,t=50,b=0))
fig6.show()